### Make sure to switch to the `generative_models` kernel from above!

Depending on your application, different storage requirements may be necessary.

- Option 1: `$TMPDIR`: a temp directory created for every job, gets deleted after every job dies. It is created under `/scratch/` on the local machine
- Option 2: `/cluster/scratch/<username>`: a private scratch space, a lot of space but files get deleted every 2-3 weeks
- Option 3: `/cluster/home/<username>`: never get deleted but limited space.

In [ ]:
import torch
# make sure that torch version is 2.0.0+cu117
print(torch.__version__)


In [ ]:
!pip install diffusers accelerate transformers mediapy triton scipy ftfy spacy==3.4.4

In [ ]:
# here we will start with option 1
import os

BASE_PATH = f"{os.getenv('TMPDIR')}/generative_models"

if not os.path.exists(BASE_PATH):
    os.makedirs(BASE_PATH)

In [ ]:
import mediapy as media
import torch
from diffusers import StableDiffusionPipeline


model_id = "stabilityai/stable-diffusion-2-1"

device = "cuda"

pipe = StableDiffusionPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    revision=None,
    cache_dir=BASE_PATH,
)
pipe.enable_xformers_memory_efficient_attention()

pipe = pipe.to(device)

image_length = 768

In [ ]:
prompt = "a photo of Pikachu fine dining with a view to the Eiffel Tower"
num_images = 4

negative_prompt = None
pipe.safety_checker = None

with torch.no_grad():
    images = pipe(
        prompt,
        height = image_length,
        width = image_length,
        num_inference_steps = 25,
        guidance_scale = 9,
        num_images_per_prompt = num_images,
        negative_prompt = negative_prompt,
    ).images
    
media.show_images(images)